%md
## Create UC-Functions to access the yahoo finance API

Get access to :
https://www.financeapi.net/

Create your own API-KEY ( free key will have limitations, please check the dashboard, limk below)

Access to Yahoofinace dashboard : 
 https://www.financeapi.net/dashboard

 



In [0]:
%pip install --quiet -U databricks-sdk langchain-community langchain-openai mlflow  # Install/update specified Python packages quietly without showing output
dbutils.library.restartPython()  # Restart the Python interpreter to ensure newly installed packages are available

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.5.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.9.2 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
catalog = "sarbanimaiti_catalog"  # Define the catalog name for the database
dbName = db = "agent_demo"  # Define the database name and assign it to both dbName and db variables

In [0]:

# # Drop the function if it already exists/or use this code to clean up fucntions which are not in use
# spark.sql('DROP FUNCTION IF EXISTS sarbanimaiti_catalog.agent_demo.youtube_search;')

DataFrame[]

In [0]:
# Drop the existing function if it exists to avoid conflicts
spark.sql('DROP FUNCTION IF EXISTS sarbanimaiti_catalog.agent_demo.finance_insight_api;')

# Create or replace the Unity Catalog function for financial insights
spark.sql('''
CREATE OR REPLACE FUNCTION sarbanimaiti_catalog.agent_demo.finance_insight_api (query STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Returns financial insights for a given ticker symbol.'
AS
$$
try:
    # Import requests module to make HTTP requests
    import requests
    # Define the API endpoint for financial insights
    url = "https://yfapi.net/ws/insights/v1/finance/insights"

    # Set the query parameter with the symbol provided to the function
    params = {"symbol": query}

    # Include your API key in the request headers for authentication
    headers = {
        'x-api-key': "<your own api-key>"
    }

    # Make a GET request to the API
    response = requests.request("GET", url, headers=headers, params=params)

    # Initialize an empty list to hold summaries
    summaries = []
    # Parse the JSON response
    data = response.json()
    # Control variable to ensure only one summary is added
    cntrol = 1

    # Iterate through the reports in the response
    for report in data['finance']['result']['reports']:
        # Check if the report is from Argus Research and control is 1
        if report['provider'] == "Argus Research" and cntrol ==1:
            # Append the summary to the summaries list
            summaries.append(report['summary'])
            # Update control to prevent adding more summaries
            cntrol = cntrol + 1

    # Return the list of summaries
    return summaries
except Exception as e:
    # Return an error message if an exception occurs
    return "Error calling YouTube Search API: {{e}}".format(e=str(e))
$$;
''')

DataFrame[]

In [0]:
# Drop the existing function to avoid conflicts
spark.sql('DROP FUNCTION IF EXISTS sarbanimaiti_catalog.finance_stock_quote.finance_api;')

# Define a new function in the Unity Catalog
spark.sql('''
CREATE OR REPLACE FUNCTION sarbanimaiti_catalog.agent_demo.finance_stock_quote (query STRING)
RETURNS STRING
LANGUAGE PYTHON
COMMENT 'Returns financial summary of ticker.'
AS
$$
try:
    # Import requests to enable HTTP calls
    import requests
    # API endpoint for fetching finance quotes
    url = "https://yfapi.net/v6/finance/quote"
    # Parameters for the API call, specifying the stock symbols
    params = {"symbols": query}
    # Headers for the API call, including the API key for authentication
    headers = {
        'x-api-key': "<your own api-key>"
    }
    # Execute the GET request with specified URL, headers, and parameters
    response = requests.request("GET", url, headers=headers, params=params)
    # Return the JSON response from the API
    return response.json()
except Exception as e:
    # Return an error message if an exception occurs
    return "Error calling YouTube Search API: {{e}}".format(e=str(e))
$$;
''')

DataFrame[]

#### Test functions using notebook in case you want to troubleshoot the uc-function calling output or agent behaviour

Otherwise skip the belwo steps

In [0]:

import requests

#url = "https://yfapi.net/ws/insights/v1/finance/insights"
url = "https://yfapi.net/v6/finance/quote" 

query = "AAPL"

params = {"symbols": query}

headers = {
    'x-api-key': "<your own api-key>"
    }

response = requests.request("GET", url, headers=headers, params=params)

print(response.json())

{'quoteResponse': {'result': [{'language': 'en-US', 'region': 'US', 'quoteType': 'EQUITY', 'typeDisp': 'Equity', 'quoteSourceName': 'Nasdaq Real Time Price', 'triggerable': True, 'customPriceAlertConfidence': 'HIGH', 'currency': 'USD', 'regularMarketTime': 1727380801, 'regularMarketDayHigh': 228.5, 'regularMarketDayRange': '225.41 - 228.5', 'regularMarketDayLow': 225.41, 'regularMarketVolume': 36490267, 'regularMarketPreviousClose': 226.37, 'bid': 227.37, 'ask': 238.68, 'bidSize': 1, 'epsTrailingTwelveMonths': 6.56, 'epsForward': 7.48, 'epsCurrentYear': 6.7, 'priceEpsCurrentYear': 33.95821, 'sharesOutstanding': 15204100096, 'bookValue': 4.382, 'fiftyDayAverage': 222.0196, 'fiftyDayAverageChange': 5.5003967, 'fiftyDayAverageChangePercent': 0.024774374, 'twoHundredDayAverage': 197.008, 'twoHundredDayAverageChange': 30.512009, 'twoHundredDayAverageChangePercent': 0.154877, 'marketCap': 3459237019648, 'forwardPE': 30.417112, 'priceToBook': 51.921497, 'sourceInterval': 15, 'exchangeDataDela

In [0]:

import requests

url = "https://yfapi.net/ws/insights/v1/finance/insights"
#url = "https://yfapi.net/v6/finance/quote" 

query = "AAPL"

params = {"symbol": query}

headers = {
    'x-api-key': "<your own api-key>"
    }

response = requests.request("GET", url, headers=headers, params=params)

print(response.json())

{'finance': {'result': {'symbol': 'AAPL', 'instrumentInfo': {'technicalEvents': {'provider': 'Trading Central', 'shortTerm': 'up', 'midTerm': 'up', 'longTerm': 'up'}, 'keyTechnicals': {'provider': 'Trading Central', 'support': 173.57, 'resistance': 229.4, 'stopLoss': 215.087401}, 'valuation': {'color': 0.0, 'description': 'Overvalued', 'discount': '3%', 'relativeValue': 'Premium', 'provider': 'Trading Central'}, 'recommendation': {'targetPrice': 250.0, 'provider': 'Argus Research', 'rating': 'BUY'}}, 'reports': [{'id': '41262_Technical Analysis_1726744358000', 'title': 'Technical Assessment: Bullish in the Intermediate-Term', 'provider': 'Argus Research', 'publishedOn': '2024-09-19T11:12:38Z', 'summary': "The Federal Reserve cut the fed funds rate by 50 basis points (bps), energizing buyers for about 45 minutes yesterday. Then, as often happens, investors faded the first move, stocks were lower into the close, and currently the futures market is through the roof. End of day, the Fed's 

In [0]:

print(response.json())

{'finance': {'result': {'symbol': 'AAPL', 'instrumentInfo': {'technicalEvents': {'provider': 'Trading Central', 'shortTerm': 'up', 'midTerm': 'up', 'longTerm': 'up'}, 'keyTechnicals': {'provider': 'Trading Central', 'support': 173.57, 'resistance': 229.4, 'stopLoss': 215.087401}, 'valuation': {'color': 0.0, 'description': 'Overvalued', 'discount': '3%', 'relativeValue': 'Premium', 'provider': 'Trading Central'}, 'recommendation': {'targetPrice': 250.0, 'provider': 'Argus Research', 'rating': 'BUY'}}, 'reports': [{'id': '41262_Technical Analysis_1726744358000', 'title': 'Technical Assessment: Bullish in the Intermediate-Term', 'provider': 'Argus Research', 'publishedOn': '2024-09-19T11:12:38Z', 'summary': "The Federal Reserve cut the fed funds rate by 50 basis points (bps), energizing buyers for about 45 minutes yesterday. Then, as often happens, investors faded the first move, stocks were lower into the close, and currently the futures market is through the roof. End of day, the Fed's 

In [0]:
summaries = []
data = response.json()
cntrol = 1

for report in data['finance']['result']['reports']:
    
    if report['provider'] == "Argus Research" and cntrol ==1:
        summaries.append(report['summary'])
        cntrol = cntrol + 1

# # Print summaries
# for i, summary in enumerate(summaries, 1):
#     print(f"Summary {i}: {summary}")

In [0]:
summaries

["The Federal Reserve cut the fed funds rate by 50 basis points (bps), energizing buyers for about 45 minutes yesterday. Then, as often happens, investors faded the first move, stocks were lower into the close, and currently the futures market is through the roof. End of day, the Fed's target is now 4.75%-5.25%. By the last 2024 meeting on December 18, there is a 50% probability that the funds range will be 4.00%-4.25% and a 32% chance the target will be 4.25%-4.50%. Treasury yields across the curve ticked higher, mostly in the five- and 10-year yields, which rose 30 to 40 bps. Market yields have been falling since April in anticipation of a slowing economy and the rate.   We are now in the last half of September, and the next two weeks historically are the worst of the year for stocks. Did we front-load those losses during the first four trading days of the month or does the market have pain ahead? Short term, it's hard to get a good feel for what will play out. But we feel confident 